# Tutorial - Remote Arrow

This notebook serves as an interactive tutorial (and documentation) to **Remote Arrow**.
More broadly it introduces you to **Apache Arrow**, specifically pyArrow and **Apache Arrow Flight**.

**Apache Arrow** is a language-independent columnar memory format for flat and hierarchical data. <br>

**Apache Arrow Flight** is a framework for high-speed transmission of Apache Arrow Tables over a network.  <br>
It achieves that by eliminating the overhead for (de)serialization of the data, as well as allowing parallel down and uploads of data chunks.<br>

**RemoteArrow** is a custom-framework built on top of Apache Arrow Flight and aims to solve the problem of redundancy <br>
and performance bottlenecks when handling shared and large dataset by outsourcing them to a centralized remote server. <br>
In other words, it tries to reduce the cost of data transmission, query, computation and storage of shared data for big data / ML applications. <br>


You can learn more by reading through the README and sources listed below:
#### Sources:

1. [Apache Arrow](https://arrow.apache.org/ "Apache Arrow")
2. [Introducing Apache Arrow Flight](https://arrow.apache.org/blog/2019/10/13/introducing-arrow-flight/ "Introducing Apache Arrow Flight") 
3. [Apache Arrow Flight API](https://arrow.apache.org/docs/format/Flight.html "Apache Arrow Flight API") 
4. [Benchmarking Apache Arrow Flight (Tanveer Ahmad, Zaid Al Ars and H. Peter Hofstee)](https://arxiv.org/pdf/2204.03032.pdf) 


## Arrow Table
Let's take a look at **the Arrow Table** format by loading a CSV file:

In [1]:
import pyarrow as pa
import pyarrow.csv as csv

table = csv.read_csv('hawaii_covid.csv')
print(table.schema)

case_month: string
res_state: string
state_fips_code: int64
res_county: string
county_fips_code: int64
age_group: string
sex: string
race: string
ethnicity: string
case_positive_specimen_interval: int64
case_onset_interval: int64
process: string
exposure_yn: string
current_status: string
symptom_status: string
hosp_yn: string
icu_yn: string
death_yn: string
underlying_conditions_yn: string


Arrow Tables support a variety of common functions for handling tabular data, such as **join**, **sort_by**, **group_by**, **drop**, **select** or **take**:

In [2]:
table.select([5]) # Get column at index 5

pyarrow.Table
age_group: string
----
age_group: [["18 to 49 years","0 - 17 years","18 to 49 years","18 to 49 years","18 to 49 years",...,"50 to 64 years","18 to 49 years","65+ years","18 to 49 years","50 to 64 years"],["18 to 49 years","65+ years","18 to 49 years","50 to 64 years","18 to 49 years",...,"18 to 49 years","0 - 17 years","0 - 17 years","18 to 49 years","18 to 49 years"],...,["50 to 64 years","50 to 64 years","50 to 64 years","50 to 64 years","50 to 64 years",...,"18 to 49 years","18 to 49 years","18 to 49 years","18 to 49 years","18 to 49 years"],["18 to 49 years","18 to 49 years","18 to 49 years","18 to 49 years","18 to 49 years",...,"50 to 64 years","50 to 64 years","50 to 64 years","50 to 64 years","50 to 64 years"]]

In [3]:
table.num_rows

199550

If you're familiar with **pandas** and/or **SQL** it may come to you as a relieve that **Arrow Table** provides similar methods and APIs. <br>
Also you can always, convert Arrow Tables into pandas dataframes.

In [4]:
table.take([0, 1, 2]).to_pandas() # Get row 0, 1, and 2 and conver to pandas df

,case_month,res_state,state_fips_code,res_county,county_fips_code,age_group,sex,race,ethnicity,case_positive_specimen_interval,case_onset_interval,process,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn,underlying_conditions_yn
0,2022-02,HI,15,HONOLULU,15003.0,18 to 49 years,Female,Asian,Hispanic/Latino,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,
1,2022-01,HI,15,MAUI,15009.0,0 - 17 years,Female,Multiple/Other,Hispanic/Latino,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,
2,2021-12,HI,15,NA,NaN,18 to 49 years,Female,Asian,NA,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,


So far, the data and queries have only been handled locally. <br> 
Let's see how we can make them accessible to other clients.

## Starting the server
Remote Arrow uses a centralized server in order to handle datasets and exposes a set of commands for uploads, downloads, queries to all clients in the network. <br>
The API is built on top of gRPC and language agnostic. That is, you can develop clients in any language <br>
or through any application as long as it follows the **Apache Arrow Flight API** [4]. <br>

In this tutorial, we are going to use Python to demonstrate the client and server interactions.
Let's start the server.

In [5]:
import subprocess
import time

SERVER_IP = "localhost"
SERVER_PORT = "5005"

server_process = subprocess.Popen(["python3","server.py","--host", SERVER_IP,"--port", SERVER_PORT])
pid = server_process.pid
time.sleep(2) # Wait for server to boot before running other cells

The server is running in the background (at localhost:5005) as long as the notebook is active. <br>
Of course, you can also run it in a separate terminal if you want to read the console output or change the host and port:

`python3 server --host <HOST> --port <PORT>`

For now, let's connect a client to the server and upload a local file:

In [6]:
from RemoteArrow import RemoteDataset
client_A = RemoteDataset("hawaii_covid.csv", hostname=SERVER_IP+":"+SERVER_PORT)

Created RemoteDataset 'hawaii_covid' on server: localhost:5005
Total rows:  199550


Here, we have instantiated a **RemoteDataset** object with path to the local CSV file. <br>
Under the hood, our object (**client_A**) converted that file into an **Arrow Table** and uploaded it to the server.

The dataset now resides on the server and is available to all clients as a **Flight** with **ID 0**. <br>

In [7]:
client_A.list_flights()


===== Flights======
[ID]	[TYPE]	[DESCRIPTION]
0 	FILE	hawaii_covid



You will also find that the server has stored that file in the */datasets* subdirectory in **.parquet** format. <br>

Instantiating **client_A** as a **RemoteDataset** also overrides the **Arrow Table methods** with corresponding RPC calls to our server. <br>
In other words, calling methods like **take**, or **select** as above, will not locally compute the query, but command the server to do the query for us, and return the result.

In [8]:
query_result = client_A.take([0,1,2]) # Will be executed on the server
query_result.to_pandas()

,case_month,res_state,state_fips_code,res_county,county_fips_code,age_group,sex,race,ethnicity,case_positive_specimen_interval,case_onset_interval,process,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn,underlying_conditions_yn
0,2022-02,HI,15,HONOLULU,15003.0,18 to 49 years,Female,Asian,Hispanic/Latino,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,
1,2022-01,HI,15,MAUI,15009.0,0 - 17 years,Female,Multiple/Other,Hispanic/Latino,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,
2,2021-12,HI,15,NA,NaN,18 to 49 years,Female,Asian,NA,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,


In [9]:
# Query results are made available as new Flights
client_A.list_flights()


===== Flights======
[ID]	[TYPE]	[DESCRIPTION]
0 	FILE	hawaii_covid
1 	CMD	 <Flight 0>.take([0, 1, 2])



Now, let's pretend to be another client (**client_B**) and connect to one of the flights.

In [10]:
client_B = RemoteDataset()

No file path or flight_id given. Connect to one of the available flights:

===== Flights======
[ID]	[TYPE]	[DESCRIPTION]
0 	FILE	hawaii_covid
1 	CMD	 <Flight 0>.take([0, 1, 2])



In [11]:
client_B.inspect(0)
client_B.connect(0)

Inspecting Flight  0
0 	FILE	hawaii_covid
Total rows: 199550
Total bytes: 42030808
Number of endpoints: 1
Schema:
case_month: string
res_state: string
state_fips_code: int64
res_county: string
county_fips_code: int64
age_group: string
sex: string
race: string
ethnicity: string
case_positive_specimen_interval: int64
case_onset_interval: int64
process: string
exposure_yn: string
current_status: string
symptom_status: string
hosp_yn: string
icu_yn: string
death_yn: string
underlying_conditions_yn: string
---
Successfully connected to flight  0


In [12]:
client_B.take([0,1,2,3,4,5]).to_pandas()

,case_month,res_state,state_fips_code,res_county,county_fips_code,age_group,sex,race,ethnicity,case_positive_specimen_interval,case_onset_interval,process,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn,underlying_conditions_yn
0,2022-02,HI,15,HONOLULU,15003.0,18 to 49 years,Female,Asian,Hispanic/Latino,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,
1,2022-01,HI,15,MAUI,15009.0,0 - 17 years,Female,Multiple/Other,Hispanic/Latino,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,
2,2021-12,HI,15,NA,NaN,18 to 49 years,Female,Asian,NA,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,
3,2022-01,HI,15,HAWAII,15001.0,18 to 49 years,Female,Multiple/Other,Hispanic/Latino,NaN,0.0,Missing,Missing,Laboratory-confirmed case,Symptomatic,Missing,Missing,Missing,
4,2022-02,HI,15,HONOLULU,15003.0,18 to 49 years,Female,Asian,Hispanic/Latino,NaN,NaN,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,
5,2022-01,HI,15,MAUI,15009.0,0 - 17 years,Female,Multiple/Other,Hispanic/Latino,NaN,0.0,Missing,Missing,Laboratory-confirmed case,Symptomatic,Missing,Missing,Missing,


Note that, **client_B** can now query against the original dataset (**hawaii_covid**) without downloading it. <br>
If we wanted to do so, we can use **fetch()**.

In [13]:
ds = client_B.fetch() # Download flight

# Convert it to pandas dataframe and continue to process it locally
df = ds.to_pandas()
print("Original num of rows: ", len(df))

dropped = df.dropna()
print("Num of rows without nulls: ", len(dropped))

dropped["case_month"]

Original num of rows:  199550
Num of rows without nulls:  7


56964     2021-12
77199     2021-12
83276     2021-10
83618     2022-05
97821     2021-08
115257    2020-12
121140    2021-12
Name: case_month, dtype: object

As you can see, there is a lot of flexibility involved in processing local and shared datasets.<br>

# Actions

**Apache Arrow Flight** provides another interface to interact with Flights, so-called **Actions**. <br>
Actions are application-specific. Our server provides Actions for **saving, deleting, clearing flights** and **shutting down the server**. <br>

Any client can send actions as well as accessing the flights as shown above. However here we are using a new client **(client_C)**.

In [14]:
client_C = RemoteDataset()
client_C.list_actions()

No file path or flight_id given. Connect to one of the available flights:

===== Flights======
[ID]	[TYPE]	[DESCRIPTION]
0 	FILE	hawaii_covid
1 	CMD	 <Flight 0>.take([0, 1, 2])
2 	CMD	 <Flight 0>.take([0, 1, 2, 3, 4, 5])


Actions
Type: clear
Description: Clear (delete) all flights. Parquet files are not affected. 
---
Type: shutdown
Description: Shut down this server.
---
Type: delete
Description: Delete a flight and corresponding .parquet file, if existing. [action.body == "<FLIGHT_ID>"]
---
Type: save
Description: Save a flight as .parquet file. [action.body == "<FLIGHT_ID> <FILENAME>"]
---


In [15]:
client_C.action(type="save", body="1 my_new_dataset") # Request server to save Flight 1 as "my_new_dataset.parquet")

Running action save
Got result:  Successfully saved flight 1 as my_new_dataset


In [16]:
client_C.action("clear") # Clear all flights (not affecting .parquet files)

Running action clear
Got result:  Successfully cleared all flights


In [17]:
client_C.list_flights()


===== Flights======
[ID]	[TYPE]	[DESCRIPTION]



Let's see what happens if our server crashes or reboots gracefully.

In [18]:
client_C.action("shutdown")
time.sleep(5) # Wait for server to properly shutdown before running other cells

Running action shutdown
Got result:  Shutdown!
Server online:  grpc+tcp://localhost:5005
Saved dataset as: /home/jovyan/datasets/hawaii_covid.parquet
Saved dataset to: /home/jovyan/datasets/my_new_dataset.parquet
Server is shutting down...


In [19]:
# Restart the server
server_process = subprocess.Popen(["python3","server.py","--host", SERVER_IP,"--port", SERVER_PORT])
pid = server_process.pid

time.sleep(2) # Wait for server to boot
client_D = RemoteDataset()

No file path or flight_id given. Connect to one of the available flights:

===== Flights======
[ID]	[TYPE]	[DESCRIPTION]
0 	FILE	hawaii_covid
1 	FILE	my_new_dataset



You should see that the server recovered the saved datasets. <br>
As a final step in this tutorial, we can delete them via the "delete" actions. The corresponding .parquet files will also be wiped from the server.

In [20]:
client_D.list_actions()
client_D.action("delete", "0")
client_D.action("delete", "1")

client_D.list_flights()


Actions
Type: clear
Description: Clear (delete) all flights. Parquet files are not affected. 
---
Type: shutdown
Description: Shut down this server.
---
Type: delete
Description: Delete a flight and corresponding .parquet file, if existing. [action.body == "<FLIGHT_ID>"]
---
Type: save
Description: Save a flight as .parquet file. [action.body == "<FLIGHT_ID> <FILENAME>"]
---
Running action delete
Got result:  Successfully deleted flight 0
Running action delete
Got result:  Successfully deleted flight 1

===== Flights======
[ID]	[TYPE]	[DESCRIPTION]
0 	N/A	This flight has been deleted.
1 	N/A	This flight has been deleted.



Note that, the IDs did not disappear and were not rearranged, inorder to maintain consistency throughout the runtime of the server.

## Conclusion
In this tutorial, we have examined how **Remote Arrow** can help to outsource the burden of computation, storage and shared access of data on a single server. <br>
Though still in its prototype phase, it can offer tremendous flexibility for managing shared tabular data.<br>
This effect becomes more evident when datasets are used in a network of heterogenous applications that work in different languages, <br>
even more so when the shared data is extremely large and/or the clients low in computational prowess.

Another aspect, we have not discussed in-depth yet is the remarkable boost in transmission speed, that has been benchmarked by *Ahmad et al.* [4]. <br>
Their publication cites that *"Flight is able to achieve up to **6000 MB/s and 4800 MB/s throughput**"* <br>
and companies like **Dremio** that use Arrow Flight **perform 20x and 30x better** compared to traditional connection methods. <br>
That means Arrow Flight becomes an even better choice for applications that require long-haul communication over the internet. <br>

Regardless of the use-case, hopefully you have learned that **Arrow** especially in combination with **Arrow Flight** provides a highly performant framework for working with big data.
